### Imports

In [15]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [16]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [17]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [18]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [19]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [20]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-75    6.1   77     0.87   0        Ruim     1.00      
-54    2.7   19     0.41   1        Bom      0.97      
-81    4.8   127    0.89   0        Ruim     1.00      
-56    2.3   14     0.41   1        Bom      0.98      
-79    4.8   18     0.85   0        Ruim     1.00      
-61    1.5   6      0.27   1        Bom      0.99      
-66    4.5   79     0.87   0        Ruim     1.00      
-49    1.8   6      0.40   1        Bom      0.99      
-95    3.8   148    0.34   0        Ruim     1.00      
-55    1.6   11     0.29   1        Bom      0.99      
-49    1.1   7      0.55   1        Bom      0.99      
-72    6.3   99     0.79   0        Ruim     1.00      
-95    9.8   79     0.92   0        Ruim     1.00      
-82    3.9   34     0.69   0        Ruim     1.00      
-46    1.0   17     0.24   1        Bom      1.00      
-88    2.7   69     0.66   0        Ruim     1.0

### Save

In [21]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
